# 全卷积神经网络  
fully convolutional network，FCN  
通过使用转置卷积，全卷积网络将中间层特征图的高和宽变换回输入图像的尺寸

In [1]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

模型构造：
* 基本的卷积神经网络提取特征  
* 1x1卷积将通道数变换为类别个数  
* 最终转置卷积将特征图的高和宽变化为输入图像尺寸，因此，模型输出与输入图像高宽相同，并且最终输出的通道包含该空间位置像素类别的预测。  
![最简单的可以用于分割的模型](../img/fcn.svg) . 
使用在ImageNet数据集上预训练的ResNet-18模型来提取图像特征，并将该网络记为pretrained_net。 ResNet-18模型的最后几层包括全局平均汇聚层和全连接层，然而全卷积网络中不需要它们

In [2]:
pretrained_net = torchvision.models.resnet18(pretrained=True)
list(pretrained_net.children())[-3:]
#children就是吧所有的层拿出来仔细看看，这边我们选取了最后三层

[Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (downsample): Sequential(
       (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
       (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (1): BasicBlock(
     (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): Batc

全卷积网络net:它复制了ResNet-18中大部分的预训练层，除了最后的全局平均汇聚层和最接近输出的全连接层。

In [3]:
#建立一个全卷积网络的实例，把除了最后一层的全部拿出来
net = nn.Sequential(*list(pretrained_net.children())[:-2])
# 给定高度为320和宽度为480的输入，net的前向传播将输入的高和宽减小至原来的1/32 
X = torch.rand(size=(1, 3, 320, 480))
net(X).shape

/root/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([1, 512, 10, 15])

图像分割数据集的输入channel给定之后经过预训练的网络，得到了512, 10, 15这样的输出  
设计能够实现图像分割任务的输出：  
* 1x1的卷积将数据集的通道数转换成21类  
* 利用转置卷积，将10，15变换回320，480（已知输入输出，求解卷积核的w、padding、stride）  
根据转置卷积的计算公式，我们已经知道，要放大32倍，那么就把步幅设为32，卷积核至少就要64（能够1/2的尺寸填满这个步长）由于最外面回多出来一圈，所以padding给32/2，这样正好就能在上下/左右消除一个32  
一个选取padding的原则就是，要保持size不变的话，padding就是kernel_size/2（偶数）、(kernel_size - 1)/2（奇数）

In [1]:
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                    kernel_size=64, padding=16, stride=32))

NameError: name 'net' is not defined